Creation des fichiers csv

In [2]:
#Importation des modules nécessaires
import pandas as pd
import os
import numpy as np
import flightradar24

#Variable servant à l'ensemble du code
fr = flightradar24.Api() #API
chemin="./data/" #Chemin vers les fichiers csv

### Récupération des vols aériens + compagnies <a class="anchor" id="chapter1"></a>

#### Définition des fonctions  <a class="anchor" id="section_1_1"></a>


In [3]:
#Fonctions nécessaires à la création des listes de vols
def liste_compagnies():
  '''
  Liste toutes les compagnies aériennes
  Retour : Liste de dictionnaire (cle : Name, code, ICAO, valeurs : valeurs associées)
  '''
  compagnies = fr.get_airlines()
  compagnies = compagnies['rows']
  return compagnies

def vols_une_compagnie(id_compagnie):
  '''
  Recherche tous les vols pour une compagnie à l'instant T
  Parametre : (string) id_compagnie -> Code ICAO de la compagnie aérienne
  Retour : Dictionnaire (cle : code vol, valeur : liste de valeurs caractérisant le vol)
  '''
  dictVol = fr.get_flights(id_compagnie)
  del dictVol['full_count']
  del dictVol['version']
  del dictVol['stats']
  return dictVol

def conv_dictVol_to_listVol (dictVol):
  '''
  Convertion des vols sous forme de dictionnaire en liste
  Parametre : (dict) dictVol -> Dictionnaire des vols pour une compagnie (code : string) : (Valeurs : list)
  Retour : Liste de Liste
  '''
  list_vols = []
  for value in dictVol.values():
    value.pop(17)
    list_vols.append(value)
  return list_vols

def vols_liste_compagnies(list_ICAO):
  '''
  Retourne tous les vols des compagnies présentes dans list_ICAO
  Parametre : (list) list_ICAO -> Liste de code ICAO pour compagnie aérienne
  Retour : Liste de dictionnaire (cle : Caractéristique du vol (string), valeur : Valeur associée (float ou string)
  '''
  list_vols_all=[]

  for icao in list_ICAO: 
    list_vols_all += conv_dictVol_to_listVol(vols_une_compagnie(icao))
  return list_vols_all

In [4]:
#Nom de colonne des caractéristiques d'un vol
key_list = ['ICAO24', 'Latitude', 'Longitude','Track', 'Altitude', 'Vitesse', 'Squawk', 'Radar_Feeder', 'Type_avion', 'Registration','Temps_UNIX', 'Depart', 'Arrivee', 'Num_IATA', 'On_ground','Vertical_Speed','Num_ICAO', 'Code_ICAO']

#### Exécution <a class="anchor" id="section_1_2"></a>


In [5]:
#Définition des données
df_compagnies = pd.DataFrame(liste_compagnies())

allCompagnies = False
list_ICAO = []

#Récupération des vols via l'API flightRadar
if(allCompagnies):
    #Pour toutes les compagnies    
    for x in df_compagnies.index:
        list_ICAO.append(df_compagnies['ICAO'][x])
else:
    #Pour une liste de compagnies
    list_ICAO = ['AFR','KLM','ACA','DLH']

#Mise sous forme de DF
df_vols = pd.DataFrame(data = vols_liste_compagnies(list_ICAO), columns=key_list)
#df_vols

#### Gestion des doublons <a class="anchor" id="section_1_3"></a>

In [6]:
#Creation de fichier si non existant
if(not(os.path.isfile(chemin+'Vols_SansDoublons.csv'))):
    pd.DataFrame(columns=key_list).to_csv(chemin+"Vols_SansDoublons.csv", index=False)

#On concatène les anciens vols aux nouveaux
df_concat = pd.concat([df_vols,pd.read_csv(chemin+"Vols_SansDoublons.csv")])

#Suppression des vols en double + vols n'ayant pas d'arrivée
df_vols = df_concat.drop_duplicates(subset=['Num_ICAO']).replace("",np.NaN).dropna(subset=['Arrivee'])

#### Export en csv <a class="anchor" id="section_1_4"></a>

In [7]:
df_vols.to_csv(chemin+"Vols_SansDoublons.csv", index=False)

### Récupération des compagnies <a class="anchor" id="chapter2"></a>

In [8]:
df_compagnies = pd.DataFrame(liste_compagnies())
df_compagnies.rename({"ICAO" : "Code_ICAO"}, axis=1, inplace=True)
df_compagnies.to_csv(chemin+"compagniesANLO.csv", index=False)

### Récupération des aéroports <a class="anchor" id="chapter2"></a>

In [11]:
#Récupération des aéroports via l'API flight radar
df_airport = pd.DataFrame(fr.get_airports()['rows'])

#Récupération des aéroports via un fichier csv
df_airportOpenFlight = pd.read_csv(chemin+"airportsOpenFlights.csv")

#On ajoute la colonne 'city' dans notre df_airport
for i in range(len(df_airport)):
    dfTmp = df_airportOpenFlight[df_airportOpenFlight['icao'] == df_airport.loc[i,'icao']]
    #Si l'aéroport existe dans df_airportOpenFlight
    if(not(dfTmp.empty)):
        df_airport.loc[i,'city'] = dfTmp.iat[0,2]


#On libère les dataframes inutiles pour la suite
df_airport.to_csv(chemin+"aeroportsANLO.csv",index=False)

### Récupération des zones géographiques <a class="anchor" id="chapter3"></a>

In [10]:
#Récupération des zones via l'API
zones = fr.get_zones()

#Mise sous forme de DF
del zones['version']
df_zones = pd.DataFrame.from_dict(zones,orient='index')

#Ajout des "sous-dictionnaire" dans la dataframe
for key, value in zones.items():
    for key2, value2 in value.items():
        if(key2 == 'subzones'):
            df_zones = pd.concat([df_zones,pd.DataFrame.from_dict(value2,orient='index')])

#Mise en forme
df_zones.drop('subzones', axis=1, inplace=True)
#Renommage des colonnes
df_zones.rename(columns={"tl_y" : "latSup", "tl_x" : "lonInf", "br_y" : "latInf", "br_x" : "lonSup"}, inplace=True)
#Rennomage des lignes + suppression d'une ligne inutile
df_zones.rename({"na_n" : "Canada", "na_c" : "USA"}, inplace=True)
df_zones.drop('na_s', inplace=True)
df_zones.drop('ceur', inplace=True)

#Ajout de valeurs 
df_zones.loc['monde'] = [80,-180,-60,180] 

#Mise en forme de l'index puis export
df_zones['Zone'] = df_zones.index
df_zones.reset_index(drop=True, inplace=True)
df_zones.to_csv(chemin+"zonesANLO.csv", index=False)
